In [1]:
import os, json, ast
import openai
import pandas as pd
from IPython.display import display, JSON

In [2]:
# some utility methods
def search_in(foo, for_string):
    methods = dir(foo)
    return [item for item in methods if for_string.lower() in item.lower()]

In [5]:
# since we are going to use openai's api, we need the open ai key
file_path = '../OPENAI_API_KEY.txt'
with open(file_path, 'r') as file:
    openai.api_key = file.readline()

# update the prices when you update the model you want to use, https://platform.openai.com/docs/pricing
def get_openai_model_price(model_name):
    if(model_name in 'gpt-4o-mini-2024-07-18'):
        return {'name': model_name, 'input_token_price_per_million_usd' : 0.15, 'cached_input_token_price_per_million_usd' : 0.075, 'output_token_price_per_million_usd' : 0.60}
    return None

# calculate price for a response
def get_chat_completion_stats(response):
    response_json = json.loads(response.model_dump_json())
    input_tokens = response_json['usage']['prompt_tokens']
    cached_input_tokens = response_json['usage']['prompt_tokens_details']['cached_tokens']
    output_tokens = response_json['usage']['completion_tokens']

    prices = get_openai_model_price(response_json['model'])
    input_token_price_per_million_usd = prices['input_token_price_per_million_usd']
    cached_input_token_price_per_million_usd = prices['cached_input_token_price_per_million_usd']
    output_token_price_per_million_usd = prices['output_token_price_per_million_usd']

    million = 1000000
    input_tokens_cost = (input_tokens/million)*input_token_price_per_million_usd
    cached_input_tokens_cost = (cached_input_tokens/million)*cached_input_token_price_per_million_usd
    output_token_cost = (output_tokens/million)*output_token_price_per_million_usd

    total_cost = input_tokens_cost + cached_input_tokens_cost + output_token_cost
    output = {
        'input': {
            'rate': input_token_price_per_million_usd,
            'tokens': input_tokens,
            'cost': f'{input_tokens_cost:.10f}'
        },
        'cached_input': {
            'rate': cached_input_token_price_per_million_usd,
            'tokens': cached_input_tokens,
            'cost': f'{cached_input_tokens_cost:.10f}'
        },
        'output': {
            'rate': output_token_price_per_million_usd,
            'tokens': output_tokens,
            'cost': f'{output_token_cost:.10f}'
        },
        'total': {
            'cost': f'{total_cost:.10f}'
        }
    }
    return output


import ipywidgets as widgets
from IPython.display import display, HTML
def display_stats(stats):
    data = stats
    # Create an HTML formatted string for a compact table layout
    html_content = f"""
    <style>
        .usage-table {{
            font-family: Arial, sans-serif;
            font-size: 12px;
            width: 70%;
            border-collapse: collapse;
            margin: 10px auto;
        }}
        .usage-table th, .usage-table td {{
            padding: 6px 10px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        .usage-table th {{
            background-color: #007bff;
            color: white;
            text-align: center;
        }}
        .input-row {{ background-color: #e3f2fd; }}
        .cached-row {{ background-color: #f1f8e9; }}
        .output-row {{ background-color: #ffebee; }}
        .total-row {{ background-color: #007bff; font-weight: bold; }}
    </style>
    
    <table class="usage-table">
        <tr>
            <th>Type</th>
            <th>Rate ($/1M)</th>
            <th>Tokens</th>
            <th>Cost ($)</th>
        </tr>
        <tr class="input-row">
            <td>Input 🔽</td>
            <td>{data['input']['rate']}</td>
            <td>{data['input']['tokens']}</td>
            <td>{data['input']['cost']}</td>
        </tr>
        <tr class="cached-row">
            <td>Cached Input 🔄</td>
            <td>{data['cached_input']['rate']}</td>
            <td>{data['cached_input']['tokens']}</td>
            <td>{data['cached_input']['cost']}</td>
        </tr>
        <tr class="output-row">
            <td>Output 🔼</td>
            <td>{data['output']['rate']}</td>
            <td>{data['output']['tokens']}</td>
            <td>{data['output']['cost']}</td>
        </tr>
        <tr class="total-row">
            <td>Total 💰</td>
            <td>—</td>
            <td>—</td>
            <td>{data['total']['cost']}</td>
        </tr>
    </table>
    """
    
    # Display in Jupyter Notebook
    display(HTML(html_content))

def display_response(response):
    display(JSON(json.loads(response.model_dump_json())))
    stats = get_chat_completion_stats(response)
    display_stats(stats)

In [6]:
# let's make a simple openai api call to send in a prompt and get back a response
def get_chat_model_completions(messages):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        temperature = 0,
        max_tokens = 900
    )
    display_response(response)
    return response.choices[0].message.content

test_messages = [
    {'role': 'system', 'content': 'You are a joke teller, you give responses in less than 40 words'},
    {'role': 'user', 'content': 'tell me a joke'}
]

get_chat_model_completions(test_messages)

# if we see a response, it means the api-key is still working

def moderation_check(user_input):
    response = openai.moderations.create(input=user_input)
    flagged = response.results[0].flagged
    return 'Flagged' if flagged else 'Not Flagged'

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,30,0.0000045000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,18,0.0000108000
Total 💰,—,—,0.0000153000


In [7]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'low','Multitasking': 'high','Processing speed': 'high','Budget': '150000'}

    system_message = f"""

    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    The value for 'budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values.

    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}

    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}

    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n


    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_req}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation
    

In [8]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'GPU intensity','Display quality','Portability','Multitasking',' Processing speed','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number with currency.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """

    response = get_text_model_completions(prompt)
    return response

In [9]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The values for all keys, except \'budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stated by user.\n    The value for \'budget\' should be a numerical value extracted from the user\'s r

In [10]:
response = get_chat_model_completions(debug_conversation)
debug_conversation.append({'role': 'assistant', 'content': response})
print(response)

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1171,0.0001756500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,30,0.0000180000
Total 💰,—,—,0.0001936500


Hello! I'm here to help you find the perfect laptop based on your needs. Please share your requirements and how you plan to use the laptop.


In [11]:
debug_conversation.append({'role': 'user', 'content': "Hi, I am Suyash. I need a laptop for coding"})
response = get_chat_model_completions(debug_conversation)
debug_conversation.append({'role': 'assistant', 'content': response})
print(response)

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1222,0.0001833000
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,62,0.0000372000
Total 💰,—,—,0.0003069000


Great, Suyash! Coding can vary in requirements depending on the type of development work you do. Are you primarily working on web development, software development, or perhaps mobile app development? Understanding your specific focus will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now.


In [12]:
debug_conversation.append({'role': 'user', 'content': 'I do web development and software development'})
response = get_chat_model_completions(debug_conversation)
debug_conversation.append({'role': 'assistant', 'content': response})
print(response)

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1298,0.0001947000
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,105,0.0000630000
Total 💰,—,—,0.0003441000


Thank you for the clarification, Suyash! For web and software development, you'll likely need a laptop with good multitasking capabilities, as you'll be running multiple applications and possibly virtual machines. 

Additionally, a decent processing speed is important for compiling code and running applications smoothly. 

Do you also require a high-quality display for design work or debugging, or is that less of a priority for you? And how important is portability? Do you often work on the go, or do you primarily work from a fixed location?


In [13]:
debug_conversation.append({'role': 'user', 'content': "No, i don't need a high quality display. Yes i would like the laptop to be portable"})
response = get_chat_model_completions(debug_conversation)
debug_conversation.append({'role': 'assistant', 'content': response})
print(response)

<IPython.core.display.JSON object>

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1429,0.0002143500
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,116,0.0000696000
Total 💰,—,—,0.0003799500


Thank you for the information, Suyash! Based on your responses, I can infer the following:

- **Multitasking**: High (since you'll be running multiple applications)
- **Processing speed**: High (for compiling and running applications)
- **Display quality**: Low (as you mentioned it's not a priority)
- **Portability**: Medium (since you want it to be portable)

Now, could you please let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements.
